### How can we choose a proper relaxation time list in our PMMA contact problem?

Our viscoelastic solver is based on linear viscoelastic model, with widely-used generalized Maxwell model. The parameters, i.e. relaxation times and elastic moduli of generalized Maxwell model are determined from either a relaxation or a Dynamical Mechanical Analysis(DMA) experiments. In Jalocha(2015)[1], they propose two techniques to identify an optimal series of relaxation times. In the case of the relaxation experiment relaxation times will be optimized from the numerical integration of the measured relaxation spectrum. In the case of the DMA experiments we show that mathematical results obtained by Krein and Nudelmann can be used to determine the complete series of relaxation times.

The main constitutive unknown is the relaxation spectrum $H(\tau)$ (Findley et al., 1976) which is related to the relaxation modulus $E(t)$ by:

$$
E(t)=E_0+\int_{-\infty}^{\infty} H(\tau) e^{-\frac{t}{\tau}} d \ln (\tau)
$$


#### Reference 

[1] Jalocha, D., A. Constantinescu, and R. Neviere. ‘Revisiting the Identification of Generalized Maxwell Models from Experimental Results’. International Journal of Solids and Structures 67–68 (August 2015): 169–81. https://doi.org/10.1016/j.ijsolstr.2015.04.018.

[2]
